# RAG on HTML documents


## Step-1: Configuration

In [1]:
from my_config import MY_CONFIG

## Step-2: Setup Embeddings

In [2]:
# If connection to https://huggingface.co/ failed, uncomment the following path
import os
os.environ['HF_ENDPOINT'] = 'https://hf-mirror.com'

In [3]:
from llama_index.embeddings.huggingface import HuggingFaceEmbedding
from llama_index.core import Settings

Settings.embed_model = HuggingFaceEmbedding(
    model_name = MY_CONFIG.EMBEDDING_MODEL
)

/home/sujee/my-stuff/projects/ai-alliance/allycat-1/.venv/lib/python3.11/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


## Step-3: Connect to Milvus

In [4]:
# connect to vector db
from llama_index.core import VectorStoreIndex, StorageContext
from llama_index.vector_stores.milvus import MilvusVectorStore

vector_store = MilvusVectorStore(
    uri = MY_CONFIG.DB_URI ,
    dim = MY_CONFIG.EMBEDDING_LENGTH , 
    collection_name = MY_CONFIG.COLLECTION_NAME,
    overwrite=False  # so we load the index from db
)
storage_context = StorageContext.from_defaults(vector_store=vector_store)

print ("✅ Connected to Milvus instance: ", MY_CONFIG.DB_URI )

/home/sujee/my-stuff/projects/ai-alliance/allycat-1/.venv/lib/python3.11/site-packages/milvus_lite/__init__.py:15: UserWarning: pkg_resources is deprecated as an API. See https://setuptools.pypa.io/en/latest/pkg_resources.html. The pkg_resources package is slated for removal as early as 2025-11-30. Refrain from using this package or pin to Setuptools<81.
  from pkg_resources import DistributionNotFound, get_distribution
2025-07-14 00:23:38,214 [DEBUG][_create_connection]: Created new connection using: async-workspace/rag_website_milvus.db (async_milvus_client.py:599)


✅ Connected to Milvus instance:  workspace/rag_website_milvus.db


## Step-4: Load Document Index from DB

In [5]:
%%time

from llama_index.core import VectorStoreIndex

index = VectorStoreIndex.from_vector_store(
    vector_store=vector_store, storage_context=storage_context)

print ("✅ Loaded index from vector db:", MY_CONFIG.DB_URI )

✅ Loaded index from vector db: workspace/rag_website_milvus.db
CPU times: user 109 ms, sys: 16.8 ms, total: 126 ms
Wall time: 123 ms


## Step-5: Setup LLM

In [6]:
from llama_index.llms.litellm import LiteLLM

# Setup LLM
print (f"✅ Using LLM model : {MY_CONFIG.LLM_MODEL}")
Settings.llm = LiteLLM (
        model=MY_CONFIG.LLM_MODEL,
    )

✅ Using LLM model : ollama/gemma3:1b


## Step-6: Query

In [7]:
import query_utils

query_engine = index.as_query_engine()
query = query_utils.tweak_query('What is AI Alliance?', MY_CONFIG.LLM_MODEL)
res = query_engine.query(query)
print(res)

The AI Alliance is an international community of researchers, developers, and organizational leaders committed to fostering open innovation across the AI technology landscape to accelerate progress, improve safety, security, diversity and economic competitiveness in AI.


In [8]:
query_engine = index.as_query_engine()
query = query_utils.tweak_query('What are the main focus areas of AI Alliance?', MY_CONFIG.LLM_MODEL)
res = query_engine.query(query)
print(res)

The AI Alliance is focused on fostering an open community and enabling developers and researchers to accelerate responsible innovation in AI while ensuring scientific rigor, trust, safety, security, diversity and economic competitiveness.


In [9]:
query_engine = index.as_query_engine()
query = query_utils.tweak_query('What are some ai alliance projects?', MY_CONFIG.LLM_MODEL)
res = query_engine.query(query)
print(res)

Based on the provided text, here are some of the AI Alliance projects mentioned:

*   FPT Software
*   Hebrew University of Jerusalem
*   Hugging Face
*   IBM
*   Abdus Salam International Centre for Theoretical Physics (ICTP)
*   Imperial College London
*   Indian Institute of Technology Bombay
*   Institute for Computer Science, Artificial Intelligence
*   Intel
*   Keio University
*   LangChain
*   LlamaIndex
*   Linux Foundation
*   Mass Open Cloud Alliance, operated by Boston University and Harvard
*   Meta
*   Mohamed bin Zayed University of Artificial Intelligence
*   MLCommons
*   National Aeronautics and Space Administration
*   National Science Foundation
*   New York University
*   NumFOCUS
*   OpenTeams
*   Oracle
*   Partnership on AI
*   Quansight
*   Red Hat
*   Rensselaer Polytechnic Institute
*   Roadzen
*   Sakana AI
*   SB Intuitions
*   ServiceNow
*   Silo AI
*   Simons Foundation
*   Sony Group
*   Stability AI
*   Together AI
*   TU Munich
*   UC Berkeley College 

In [10]:
query_engine = index.as_query_engine()
query = query_utils.tweak_query('Where was the demo night held?', MY_CONFIG.LLM_MODEL)
res = query_engine.query(query)
print(res)

On August 8th, The AI Alliance hosted Open Source AI Demo Night in San Francisco.


In [11]:
query_engine = index.as_query_engine()
query = query_utils.tweak_query('What is the AI Alliance doing in the area of material science?', MY_CONFIG.LLM_MODEL)
res = query_engine.query(query)
print(res)

The AI Alliance is focused on developing and sharing foundational models for science.


In [12]:
query_engine = index.as_query_engine()
query = query_utils.tweak_query('How do I join the AI Alliance?', MY_CONFIG.LLM_MODEL)
res = query_engine.query(query)
print(res)

By submitting this form, you agree that the AI Alliance will collect and process the personal information you provide to keep you informed about AI Alliance initiatives and enable your involvement in AI Alliance activities. Additionally, you agree that the AI Alliance may share the personal information you provide with its member organizations so that they may communicate with you about AI Alliance initiatives and your involvement in AI Alliance activities.

You may withdraw your consent for the processing of your personal information by the AI Alliance. Please contact us to request a permanent deletion.


In [13]:
query_engine = index.as_query_engine()
query = query_utils.tweak_query('When was the moon landing?', MY_CONFIG.LLM_MODEL)
res = query_engine.query(query)
print(res)

The context does not provide information about the moon landing.
